In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
# linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

2024-03-22 14:52:47.152703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 14:52:47.152858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 14:52:47.378120: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [7]:
df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
display(df.head())
print(df.shape)

/tmp/ipykernel_33/3152433989.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


(54214, 4)


import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [8]:
df[0:1]

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...


In [9]:
df.drop(columns=['ID','TITLE'],inplace=True)

In [10]:
df.head()

,GENRE,DESCRIPTION
0,drama,Listening in to a conversation between his do...
1,thriller,A brother and sister with a past incestuous r...
2,adult,As the bus empties the students for their fie...
3,drama,To help their unemployed father make ends mee...
4,drama,The film's title refers not only to the un-re...


In [11]:
average_length = df['DESCRIPTION'].str.len().mean()

print(f"The average length of movie descriptions is: {average_length:.2f}")


The average length of movie descriptions is: 600.45


In [12]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
df['DESCRIPTION']=df['DESCRIPTION'].str.lower()

In [14]:
df.head()

,GENRE,DESCRIPTION
0,drama,listening in to a conversation between his do...
1,thriller,a brother and sister with a past incestuous r...
2,adult,as the bus empties the students for their fie...
3,drama,to help their unemployed father make ends mee...
4,drama,the film's title refers not only to the un-re...


In [15]:
import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text
df['DESCRIPTION']=df['DESCRIPTION'].apply(clean_text)
df['DESCRIPTION']

0         listening in to a conversation between his do...
1         a brother and sister with a past incestuous r...
2         as the bus empties the students for their fie...
3         to help their unemployed father make ends mee...
4         the films title refers not only to the unreco...
                               ...                        
54209     this shortlived nbc live sitcom centered on b...
54210     the next generation of exploitation the siste...
54211     ze bestaan echt is a standup comedy about gro...
54212     walter and vivian live in the country and hav...
54213     on labor day weekend 1935 the most intense hu...
Name: DESCRIPTION, Length: 54214, dtype: object

In [16]:
import nltk
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def stem_text(text):
    tokens = nltk.word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    stemmed_text = ' '.join(stemmed_tokens)
    return stemmed_text

df['DESCRIPTION'] = df['DESCRIPTION'].apply(stem_text)

In [17]:
df['DESCRIPTION']

0        listen in to a convers between hi doctor and p...
1        a brother and sister with a past incestu relat...
2        as the bu empti the student for their field tr...
3        to help their unemploy father make end meet ed...
4        the film titl refer not onli to the unrecov bo...
                               ...                        
54209    thi shortliv nbc live sitcom center on bonino ...
54210    the next gener of exploit the sister of kapa b...
54211    ze bestaan echt is a standup comedi about grow...
54212    walter and vivian live in the countri and have...
54213    on labor day weekend 1935 the most intens hurr...
Name: DESCRIPTION, Length: 54214, dtype: object

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'descriptions' is the column name containing the text data
corpus = df['DESCRIPTION']

# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the corpus
X = vectorizer.fit_transform(corpus)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, df['GENRE'], test_size=0.2, random_state=42)

In [20]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
# Train a multi-class logistic regression model
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42).fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
# Make predictions
y_pred = clf.predict(X_test)

In [23]:
y_train_score=clf.predict(X_train)

In [ ]:
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy}")

In [23]:
from sklearn.metrics import accuracy_score, classification_report

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Train Accuracy:", accuracy_score(y_train, y_tain))
print("Classification Report:\n", classification_report(y_test, y_pred))

Test Accuracy: 0.5876602416305451
Train Accuracy: 0.6984390491342141


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
                precision    recall  f1-score   support

      action        0.55      0.24      0.33       263
       adult        0.87      0.29      0.44       112
   adventure        0.44      0.14      0.21       139
   animation        0.50      0.04      0.07       104
   biography        0.00      0.00      0.00        61
      comedy        0.52      0.61      0.56      1443
       crime        0.12      0.01      0.02       107
 documentary        0.66      0.86      0.75      2659
       drama        0.54      0.79      0.65      2697
      family        0.52      0.08      0.14       150
     fantasy        0.00      0.00      0.00        74
   game-show        0.83      0.38      0.52        40
     history        0.00      0.00      0.00        45
      horror        0.66      0.57      0.61       431
       music        0.67      0.42      0.51       144
     musical        0.00      0.00      0.00        50
     mystery        1.00      0.02      

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Using naive bayes classifier

In [22]:
from sklearn.naive_bayes import MultinomialNB

In [24]:
#naive bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, y_train)

#predicted y
y_pred = naive_bayes_classifier.predict(X_test)

Using Deep learning 

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, df['GENRE'], test_size=0.2, random_state=42)


2024-03-21 15:17:00.529629: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 15:17:00.529748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 15:17:00.728050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [25]:
import tensorflow as tf
from scipy.sparse import csr_matrix

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['GENRE'], test_size=0.2, random_state=42)

# Encode string labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert the target variable to one-hot encoded format
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train_encoded, num_classes=27)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=27)

# Convert sparse matrices to CSR format
X_train_csr = csr_matrix(X_train)
X_test_csr = csr_matrix(X_test)

# Create sparse tensors from CSR matrices
X_train_sparse = tf.sparse.from_scipy_sparse(X_train_csr)
X_test_sparse = tf.sparse.from_scipy_sparse(X_test_csr)

AttributeError: module 'tensorflow._api.v2.sparse' has no attribute 'from_scipy_sparse'

In [ ]:
# Define the input shape
input_shape = X_train.shape[1]

# Define the deep learning model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(input_shape,)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(27, activation='softmax'))  # Output layer with 27 units for 27 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_sparse, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test_sparse, y_test_one_hot))

# Evaluate the model
_, accuracy = model.evaluate(X_test_sparse, y_test_one_hot)
print('Test accuracy:', accuracy)

In [ ]:
# from sklearn.decomposition import PCA
# # Create a PCA instance and specify the desired number of components
# pca = PCA(n_components=500)  # Adjust the number of components as needed

# # Fit and transform the training data
# X_train_pca = pca.fit_transform(X_train_dense)

# # Transform the test data using the trained PCA
# X_test_pca = pca.transform(X_test_dense)